## Tarea
Va a consistir de 4 pasos basados en lo que aprendimos de como manipular los INEGI shape files.

## Primero
#### Vas a escoger 5 estados y vas a graficar sus vecinos como vimos en el otro archivo. A cada uno de estos grupos los llamaremos grupos de estados.
#### Usa el shapefile de los estados


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString, Polygon
from shapely.wkt import loads
import fiona
import matplotlib.pyplot as plt

In [2]:
#Pon el path a tu shapefile de la carpeta de Estados que descargaste. Ejemplo:
file_pathAShapefile_Estados = 'ShapeFilesFiles/Estados/conjunto_de_datos/00ent.shp'
#Pon el path a tu shapefile de la carpeta de servicios_de_salud_shp que descargaste. Ejemplo:
file_pathAShapefile_ServicioSalud = 'ShapeFilesFiles/conjunto_de_datos/denue_inegi_62_.shp'
#Leer shapefiles que usaremos
gdf_mexico = gpd.read_file(file_pathAShapefile_Estados)
gdf_salud = gpd.read_file(file_pathAShapefile_ServicioSalud)

In [3]:
vecinos_dict = {}
# Iterar sobre cada estado en el dataframe
for index, row in gdf_mexico.iterrows():
    # Obtener la geometría del estado actual
    estado_geom = row['geometry']
    
    # Encontrar los estados que intersectan con el estado actual (incluye fronteras y superposiciones)
    gdf_vecinos = gdf_mexico[gdf_mexico.geometry.intersects(estado_geom)]
    
    # Eliminar el estado mismo de la lista de vecinos (si se incluye)
    gdf_vecinos = gdf_vecinos[gdf_vecinos['NOMGEO'] != row['NOMGEO']]
    
    # Obtener los nombres de los vecinos
    vecinos_nombres = gdf_vecinos['NOMGEO'].tolist()
    
    # Almacenar en el diccionario
    vecinos_dict[row['NOMGEO']] = vecinos_nombres


In [4]:
def plot_estado_y_vecinos(estado, gdf, vecinos_dict):
    # Obtener la geometría del estado de interés
    estado_geom = gdf[gdf['NOMGEO'] == estado].geometry.iloc[0]
    
    # Obtener los vecinos del estado desde el diccionario
    vecinos = vecinos_dict.get(estado, [])
    
    # Filtrar los estados vecinos
    gdf_vecinos = gdf[gdf['NOMGEO'].isin(vecinos)]
    
    # Crear una figura y un eje para el gráfico
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Graficar el estado de interés con un color especial (por ejemplo, en azul)
    gdf[gdf['NOMGEO'] == estado].plot(ax=ax, color='lightblue', edgecolor='black', label=f'{estado}')
    
    # Graficar los estados vecinos con otro color (por ejemplo, en verde)
    gdf_vecinos.plot(ax=ax, color='lightgreen', edgecolor='black', label='Vecinos')
    
    # Personalizar el gráfico
    ax.set_title(f"Estado: {estado} y sus vecinos")
    
    # Mostrar el gráfico
    plt.show()

#plot_estado_y_vecinos('México', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Campeche', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Tamaulipas', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Chihuahua', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos('Oaxaca', gdf_mexico, vecinos_dict)

## Segundo
### En cada grupo de estados vas a graficar las farmacias que se encuentran en el área, al igual que un círculo a su alrededor que tenga un radio de igual longitud que la distancia de esa farmacia a la siguiente farmacia más cercana.

In [5]:
if gdf_mexico.crs != gdf_salud.crs:
    gdf_salud = gdf_salud.to_crs(gdf_mexico.crs)

In [6]:
from shapely.ops import unary_union
import geopandas as gpd
import matplotlib.pyplot as plt

def plot_estado_y_vecinos_farmacias(estado, gdf, vecinos_dict):
    # Obtener la geometría del estado de interés
    estado_geom = gdf[gdf['NOMGEO'] == estado].geometry.iloc[0]
    
    # Obtener los vecinos del estado desde el diccionario
    vecinos = vecinos_dict.get(estado, [])
    
    # Filtrar los estados vecinos
    gdf_vecinos = gdf[gdf['NOMGEO'].isin(vecinos)]
    
    # Crear una figura y un eje para el gráfico
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Graficar el estado de interés con un color especial (por ejemplo, en azul)
    gdf[gdf['NOMGEO'] == estado].plot(ax=ax, color='lightblue', edgecolor='black', label=f'{estado}')
    
    # Graficar los estados vecinos con otro color (por ejemplo, en verde)
    gdf_vecinos.plot(ax=ax, color='lightgreen', edgecolor='black', label='Vecinos')
    
    # Filtrar las farmacias que están en el estado de interés
    gdf_farmacias_entidad = gdf_salud[gdf_salud['entidad'] == estado]
    
    # Inicializar geometría para los vecinos
    geometrias_vecinos = list(gdf_farmacias_entidad.geometry)  # Comienza con las geometrías del estado
    
    # Añadir las geometrías de las farmacias en los vecinos
    for est in vecinos:
        geometrias_vecinos.extend(gdf_salud[gdf_salud['entidad'] == est].geometry)
    
    # Unir todas las geometrías
    geometria_unida_vecinos = unary_union(geometrias_vecinos)
    
    # Crear un GeoDataFrame con la geometría unida
    gdf_farmacias_vecinos = gpd.GeoDataFrame(geometry=[geometria_unida_vecinos])
    
    # Graficar las farmacias del estado sobre el mapa (en rojo)
    gdf_farmacias_entidad.plot(ax=ax, color='red', markersize=10, label='Farmacias')
    
    # Graficar las farmacias de los vecinos (en naranja)
    gdf_farmacias_vecinos.plot(ax=ax, color='orange', alpha=0.5, label='Farmacias Vecinas')
    
    # Personalizar el gráfico
    ax.set_title(f"Estado: {estado} y sus vecinos")
    plt.show()




In [7]:
#plot_estado_y_vecinos_farmacias('México', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Campeche', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Tamaulipas', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Chihuahua', gdf_mexico, vecinos_dict)
#plot_estado_y_vecinos_farmacias('Oaxaca', gdf_mexico, vecinos_dict)

## Tercero
### Por grupo de estados debes calcular cual es la farmacia que en promedio está más alejada de el resto de farmacios.
### Debes graficar las distancias de esta farmacia al resto de las farmacias en el grupo de estados.

In [8]:
def calcular_farmacia_mas_alejada_y_graficar(grupo_estados, gdf_salud):
    # Filtrar las farmacias dentro del grupo de estados
    gdf_farmacias_grupo = gdf_salud[gdf_salud['entidad'].isin(grupo_estados)]
    
    # Asegurarnos de que las geometrías estén en un CRS métrico (como UTM)
    if gdf_farmacias_grupo.crs.to_string() != "EPSG:32614":  # Cambia 32614 por el UTM correspondiente
        gdf_farmacias_grupo = gdf_farmacias_grupo.to_crs("EPSG:32614")
    
    # Extraer las geometrías (puntos) de las farmacias
    puntos_farmacias = gdf_farmacias_grupo.geometry
    
    # Calcular la distancia promedio de cada farmacia al resto
    distancias_promedio = []
    for i, punto_origen in enumerate(puntos_farmacias):
        distancias = puntos_farmacias.apply(lambda punto_destino: punto_origen.distance(punto_destino))
        distancias_promedio.append(distancias.mean())
    
    # Identificar la farmacia con la mayor distancia promedio
    indice_mas_alejado = distancias_promedio.index(max(distancias_promedio))
    farmacia_mas_alejada = puntos_farmacias.iloc[indice_mas_alejado]
    
    # Calcular las distancias de esta farmacia al resto
    distancias_a_otros = puntos_farmacias.apply(lambda punto: farmacia_mas_alejada.distance(punto))
    
    # Graficar las distancias
    plt.figure(figsize=(10, 6))
    plt.bar(range(len(distancias_a_otros)), distancias_a_otros, color="blue", alpha=0.7)
    plt.title(f"Distancias desde la farmacia más alejada al resto en el grupo de estados")
    plt.xlabel("Farmacias")
    plt.ylabel("Distancia (metros)")
    plt.show()

In [9]:
grupo_estados = vecinos_dict['México']  # O el estado que desees analizar
calcular_farmacia_mas_alejada_y_graficar(grupo_estados, gdf_salud)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fef64c644f0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

## Cuarto
### Calcula cual es el grupo de estados con mayor distancia promedio entre sus farmacias.
### Luego grafica el país completo con el shapefile de estados y grafica el grupo de estados con mayor distancia promedio entre farmacias de un color distinto.

In [ ]:
def calcular_grupo_mayor_distancia_promedio(vecinos_dict, gdf_salud):
    # Inicializar variables para almacenar resultados
    max_distancia_promedio = 0
    grupo_con_max_distancia = None

    # Iterar por cada grupo de vecinos
    for estado, grupo_estados in vecinos_dict.items():
        # Incluir el estado mismo en su grupo
        grupo_estados = [estado] + grupo_estados
        
        # Filtrar las farmacias del grupo de estados
        gdf_farmacias_grupo = gdf_salud[gdf_salud['entidad'].isin(grupo_estados)]
        
        # Asegurarnos de que las geometrías estén en un CRS métrico
        if gdf_farmacias_grupo.crs.to_string() != "EPSG:32614":  # Cambia 32614 según el CRS métrico correspondiente
            gdf_farmacias_grupo = gdf_farmacias_grupo.to_crs("EPSG:32614")
        
        # Extraer las geometrías de las farmacias
        puntos_farmacias = gdf_farmacias_grupo.geometry
        
        # Calcular la distancia promedio entre las farmacias del grupo
        distancias_promedio = []
        for punto_origen in puntos_farmacias:
            distancias = puntos_farmacias.apply(lambda punto_destino: punto_origen.distance(punto_destino))
            distancias_promedio.append(distancias.mean())
        
        # Promediar las distancias promedio de cada farmacia
        distancia_promedio_grupo = np.mean(distancias_promedio)
        
        # Actualizar si este grupo tiene mayor distancia promedio
        if distancia_promedio_grupo > max_distancia_promedio:
            max_distancia_promedio = distancia_promedio_grupo
            grupo_con_max_distancia = grupo_estados

    return grupo_con_max_distancia, max_distancia_promedio

def graficar_pais_con_grupo(gdf_mexico, grupo_estados):
    # Crear figura y eje
    fig, ax = plt.subplots(figsize=(15, 15))
    
    # Graficar todos los estados en un color base
    gdf_mexico.plot(ax=ax, color='lightgrey', edgecolor='black', label='Otros estados')
    
    # Graficar los estados del grupo destacado
    gdf_grupo = gdf_mexico[gdf_mexico['NOMGEO'].isin(grupo_estados)]
    gdf_grupo.plot(ax=ax, color='lightblue', edgecolor='black', label='Grupo destacado')
    
    # Añadir título y leyenda
    ax.set_title('Mapa de México con el grupo de estados destacado', fontsize=16)
    ax.legend()
    
    # Mostrar el gráfico
    plt.show()
    
# Calcular el grupo con mayor distancia promedio entre farmacias
grupo_mayor_distancia, distancia_promedio = calcular_grupo_mayor_distancia_promedio(vecinos_dict, gdf_salud)

print(f"Grupo con mayor distancia promedio entre farmacias: {grupo_mayor_distancia}")
print(f"Distancia promedio: {distancia_promedio} metros")

# Graficar el país destacando el grupo con mayor distancia promedio
graficar_pais_con_grupo(gdf_mexico, grupo_mayor_distancia)
